In [1]:
import re
import sys
from operator import add

import pandas as pd
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

In [2]:
conf = SparkConf().setMaster("spark://sparklab-master:7077").setAppName("HW5")
sc = SparkContext.getOrCreate(conf=conf)
sqlContext=SQLContext(sc)

In [3]:
raw_data = sc.textFile("/home/twster/Spark/Projects/datasets/web-Google-2.txt",use_unicode=False)

In [4]:
rdd = raw_data.map(lambda x: x.split("\t"))

In [5]:
rdd.take(5)

[['0', '11342'],
 ['0', '824020'],
 ['0', '867923'],
 ['0', '891835'],
 ['11342', '0']]

In [6]:
import graphframes

In [7]:
keys = rdd.flatMap(lambda x: (x[0], x[1])).distinct()
keylist = keys.collect()

In [11]:
vertices = keys.map(lambda x: (x, )).toDF(["id"])
vertices.show()

+------+
|    id|
+------+
|378466|
|287145|
|228057|
|378465|
|287140|
|228050|
|378460|
|287143|
|287149|
|378469|
|825670|
|405531|
| 89373|
| 11875|
|825676|
|281770|
|425876|
|825679|
|425875|
|370259|
+------+
only showing top 20 rows



# (1) Given the Google web graph dataset, please output the list of web pages with the number of outlinks, sorted in descending order of the out-degrees.

In [12]:
#edge = rdd.map(lambda x: (x[0], x[1], x[0] + ":" +x[1])).toDF(["src", "dst", "relation"])
edge = rdd.map(lambda x: (x[0], x[1])).toDF(["src", "dst"])
edge.show()

+------+------+
|   src|   dst|
+------+------+
|     0| 11342|
|     0|824020|
|     0|867923|
|     0|891835|
| 11342|     0|
| 11342| 27469|
| 11342| 38716|
| 11342|309564|
| 11342|322178|
| 11342|387543|
| 11342|427436|
| 11342|538214|
| 11342|638706|
| 11342|645018|
| 11342|835220|
| 11342|856657|
| 11342|867923|
| 11342|891835|
|824020|     0|
|824020| 91807|
+------+------+
only showing top 20 rows



In [13]:
g = graphframes.GraphFrame(vertices, edge)
g.inDegrees.withColumnRenamed("id","NodeId").sort(desc("inDegree")).show()
#results = g.pageRank(resetProbability=0.15, tol=0.01)
#results = g.pageRank(resetProbability=0.0001, maxIter=20)

+------+--------+
|NodeId|inDegree|
+------+--------+
|537039|    6326|
|597621|    5354|
|504140|    5271|
|751384|    5182|
| 32163|    5097|
|885605|    4847|
|163075|    4731|
|819223|    4620|
|605856|    4550|
|828963|    4484|
|551829|    4220|
| 41909|    4219|
|558791|    4206|
|459074|    4187|
|407610|    4180|
|213432|    4084|
|765334|    4015|
|384666|    4010|
|173976|    3988|
|687325|    3956|
+------+--------+
only showing top 20 rows



# (2) Please output the inlink distribution of the top linked web pages, sorted in descending order of the in-degrees.

In [14]:
from pyspark.sql.functions import desc
g.outDegrees.withColumnRenamed("id","NodeId").sort(desc("outDegree")).show()

+------+---------+
|NodeId|outDegree|
+------+---------+
|506742|      456|
|203748|      372|
|305229|      372|
|768091|      330|
|808643|      277|
|412410|      268|
|600479|      265|
|376428|      258|
|156950|      257|
|885728|      256|
|667584|      253|
|685695|      248|
|282140|      247|
|598188|      245|
|579314|      244|
|411593|      231|
|321091|      229|
|838278|      225|
|302733|      216|
|915273|      213|
+------+---------+
only showing top 20 rows



# (3) Design an algorithm that maintains the connectivity of two nodes in an efficient way. Given a node v, please output the list of nodes that v points to, and the list of nodes that points to v.

# 